In [1]:
# Create the Skewed Pipeline
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import random, time

spark = SparkSession.builder \
    .appName("StreamPulse-TunePartitioning") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.sql.adaptive.enabled", "false") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()


In [4]:
random.seed(42)

events_data = []
for i in range(800000):
    r = random.random()
    if r < 0.50:
        merchant = "MEGA_MUSIC"
    elif r < 0.60:
        merchant = "BIG_BEATS"
    elif r < 0.65:
        merchant = "SOUND_SHOP"
    elif r < 0.80:
        merchant = f"store_{random.randint(1, 200):04d}"
    else:
        merchant = None  # 20% null merchant IDs

    events_data.append((
        f"EVT-{i+1:07d}",
        f"USR-{random.randint(1, 100000):06d}",
        merchant,
        random.choice(["Pop", "Rock", "Hip-Hop", "Jazz", "Electronic", "R&B"]),
        random.randint(15, 350),
        random.choice([True, False]),
        __builtins__.round(random.uniform(0.003, 0.01), 4),
    ))

events = spark.createDataFrame(events_data,
    ["event_id", "user_id", "merchant_id", "genre",
     "duration_sec", "completed", "revenue_per_stream"])

# Merchant details (small lookup table - 203 rows)
merchant_data = [
    ("MEGA_MUSIC", "Mega Music Inc.", "platinum", "National", 0.60),
    ("BIG_BEATS", "Big Beats Co.", "gold", "Regional", 0.55),
    ("SOUND_SHOP", "Sound Shop LLC", "silver", "Local", 0.50),
]
for i in range(1, 201):
    merchant_data.append((f"store_{i:04d}", f"Store {i}", "bronze", "Local",
                          __builtins__.round(random.uniform(0.40, 0.50), 2)))

merchants = spark.createDataFrame(merchant_data,
    ["merchant_id", "merchant_name", "tier", "coverage", "payout_rate"])

print(f"Events: {events.count()} | Merchants: {merchants.count()}")
print(f"\nKey distribution:")
events.groupBy("merchant_id") \
    .agg(count("*").alias("cnt")) \
    .orderBy(col("cnt").desc()) \
    .withColumn("pct", round(col("cnt") / 800000 * 100, 1)) \
    .show(10)

Events: 800000 | Merchants: 203

Key distribution:
+-----------+------+----+
|merchant_id|   cnt| pct|
+-----------+------+----+
| MEGA_MUSIC|399875|50.0|
|       NULL|160136|20.0|
|  BIG_BEATS| 79626|10.0|
| SOUND_SHOP| 40592| 5.1|
| store_0162|   652| 0.1|
| store_0155|   650| 0.1|
| store_0027|   649| 0.1|
| store_0100|   649| 0.1|
| store_0146|   646| 0.1|
| store_0043|   646| 0.1|
+-----------+------+----+
only showing top 10 rows


In [5]:
print("=" * 60)
print("BASELINE: Unoptimized Skewed Pipeline")
print("=" * 60)

total_start = time.time()

# Join events with merchants (skewed SortMerge join)
start = time.time()
joined = events.join(merchants, "merchant_id")
joined_count = joined.count()
t_join = time.time() - start

# Aggregation 1: Revenue by merchant (skewed groupBy)
start = time.time()
rev_by_merchant = joined.groupBy("merchant_id", "merchant_name", "tier") \
    .agg(
        sum(col("revenue_per_stream") * col("payout_rate")).alias("total_payout"),
        count("*").alias("streams"),
        avg("duration_sec").alias("avg_duration")
    )
rev_by_merchant.collect()
t_agg1 = time.time() - start

# Aggregation 2: Revenue by genre
start = time.time()
rev_by_genre = joined.groupBy("genre") \
    .agg(sum("revenue_per_stream").alias("total_rev"), count("*").alias("streams"))
rev_by_genre.collect()
t_agg2 = time.time() - start

# Aggregation 3: Top merchants by payout
start = time.time()
top_merchants = rev_by_merchant.orderBy(col("total_payout").desc()).limit(20)
top_merchants.collect()
t_agg3 = time.time() - start

baseline_total = time.time() - total_start

print(f"  Join:                {t_join:.3f}s  (matched {joined_count} rows)")
print(f"  Agg 1 (merchant):   {t_agg1:.3f}s")
print(f"  Agg 2 (genre):      {t_agg2:.3f}s")
print(f"  Agg 3 (top):        {t_agg3:.3f}s")
print(f"  TOTAL:              {baseline_total:.3f}s")
print(f"\nDropped rows (nulls): {events.count() - joined_count}")

# Baseline plan
print("\nBASELINE PLAN:")
joined.groupBy("merchant_id").agg(count("*")).explain()


BASELINE: Unoptimized Skewed Pipeline
  Join:                7.279s  (matched 639864 rows)
  Agg 1 (merchant):   7.721s
  Agg 2 (genre):      5.362s
  Agg 3 (top):        6.464s
  TOTAL:              26.826s

Dropped rows (nulls): 160136

BASELINE PLAN:
== Physical Plan ==
*(5) HashAggregate(keys=[merchant_id#9], functions=[count(1)])
+- *(5) HashAggregate(keys=[merchant_id#9], functions=[partial_count(1)])
   +- *(5) Project [merchant_id#9]
      +- *(5) SortMergeJoin [merchant_id#9], [merchant_id#14], Inner
         :- *(2) Sort [merchant_id#9 ASC NULLS FIRST], false, 0
         :  +- Exchange hashpartitioning(merchant_id#9, 8), ENSURE_REQUIREMENTS, [plan_id=546]
         :     +- *(1) Project [merchant_id#9]
         :        +- *(1) Filter isnotnull(merchant_id#9)
         :           +- *(1) Scan ExistingRDD[event_id#7,user_id#8,merchant_id#9,genre#10,duration_sec#11L,completed#12,revenue_per_stream#13]
         +- *(4) Sort [merchant_id#14 ASC NULLS FIRST], false, 0
            +

In [6]:
print("=" * 60)
print("FIX 1: Handle Null Merchant IDs")
print("=" * 60)

start = time.time()

# Separate null and non-null
events_valid = events.filter(col("merchant_id").isNotNull())
events_null = events.filter(col("merchant_id").isNull())

# Join only valid records
joined_fix1 = events_valid.join(merchants, "merchant_id")

# Handle nulls separately
null_summary = events_null.agg(
    count("*").alias("null_events"),
    sum("revenue_per_stream").alias("null_revenue")
).collect()[0]

joined_fix1.groupBy("merchant_id", "tier") \
    .agg(sum(col("revenue_per_stream") * col("payout_rate")), count("*"), avg("duration_sec")) \
    .collect()

t_fix1 = time.time() - start
print(f"  Fix 1 time: {t_fix1:.3f}s (was {baseline_total:.3f}s)")
print(f"  Null events handled separately: {null_summary['null_events']}")
print(f"  Improvement: {(1 - t_fix1/baseline_total)*100:.1f}%")


FIX 1: Handle Null Merchant IDs
  Fix 1 time: 9.612s (was 26.826s)
  Null events handled separately: 160136
  Improvement: 64.2%


In [7]:
print("=" * 60)
print("FIX 2: Broadcast Join (merchants table is small)")
print("=" * 60)

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "10485760")

start = time.time()

events_valid = events.filter(col("merchant_id").isNotNull())
joined_fix2 = events_valid.join(broadcast(merchants), "merchant_id")
joined_fix2.cache()
joined_fix2.count()

# All 3 aggregations from cache
joined_fix2.groupBy("merchant_id", "merchant_name", "tier") \
    .agg(sum(col("revenue_per_stream") * col("payout_rate")), count("*"), avg("duration_sec")) \
    .collect()
joined_fix2.groupBy("genre").agg(sum("revenue_per_stream"), count("*")).collect()
joined_fix2.groupBy("merchant_id").agg(sum(col("revenue_per_stream") * col("payout_rate"))) \
    .orderBy(desc("sum((revenue_per_stream * payout_rate))")).limit(20).collect()

t_fix2 = time.time() - start
print(f"  Fix 2 time: {t_fix2:.3f}s (was {baseline_total:.3f}s)")
print(f"  Improvement: {(1 - t_fix2/baseline_total)*100:.1f}%")

joined_fix2.explain()
joined_fix2.unpersist()


FIX 2: Broadcast Join (merchants table is small)
  Fix 2 time: 14.597s (was 26.826s)
  Improvement: 45.6%
== Physical Plan ==
InMemoryTableScan [merchant_id#9, event_id#7, user_id#8, genre#10, duration_sec#11L, completed#12, revenue_per_stream#13, merchant_name#15, tier#16, coverage#17, payout_rate#18]
   +- InMemoryRelation [merchant_id#9, event_id#7, user_id#8, genre#10, duration_sec#11L, completed#12, revenue_per_stream#13, merchant_name#15, tier#16, coverage#17, payout_rate#18], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(2) Project [merchant_id#9, event_id#7, user_id#8, genre#10, duration_sec#11L, completed#12, revenue_per_stream#13, merchant_name#15, tier#16, coverage#17, payout_rate#18]
            +- *(2) BroadcastHashJoin [merchant_id#9], [merchant_id#14], Inner, BuildRight, false
               :- *(2) Filter isnotnull(merchant_id#9)
               :  +- *(2) Scan ExistingRDD[event_id#7,user_id#8,merchant_id#9,genre#10,duration_sec#11L,completed#12,reve

DataFrame[merchant_id: string, event_id: string, user_id: string, genre: string, duration_sec: bigint, completed: boolean, revenue_per_stream: double, merchant_name: string, tier: string, coverage: string, payout_rate: double]

In [8]:
print("=" * 60)
print("FIX 3: Salted GroupBy")
print("=" * 60)

SALT = 10

start = time.time()

events_valid = events.filter(col("merchant_id").isNotNull())
joined_fix3 = events_valid.join(broadcast(merchants), "merchant_id")

# Salt the merchant_id
salted = joined_fix3.withColumn("salt", floor(rand() * SALT).cast("int")) \
    .withColumn("salted_key", concat(col("merchant_id"), lit("_"), col("salt")))

# Phase 1: Partial aggregation on salted key
partial = salted.groupBy("salted_key") \
    .agg(
        sum(col("revenue_per_stream") * col("payout_rate")).alias("partial_payout"),
        count("*").alias("partial_count"),
        sum("duration_sec").alias("partial_dur_sum"),
        first("tier").alias("tier"),
        first("merchant_name").alias("merchant_name"),
    )

# Phase 2: Final aggregation removing salt
final = partial \
    .withColumn("merchant_id", regexp_extract(col("salted_key"), "^(.+)_\\d+$", 1)) \
    .groupBy("merchant_id", "merchant_name", "tier") \
    .agg(
        sum("partial_payout").alias("total_payout"),
        sum("partial_count").alias("streams"),
        (sum("partial_dur_sum") / sum("partial_count")).alias("avg_duration"),
    )

final.collect()
t_fix3 = time.time() - start

print(f"  Fix 3 time:    {t_fix3:.3f}s (was {baseline_total:.3f}s)")
print(f"  Improvement:   {(1 - t_fix3/baseline_total)*100:.1f}%")

# Verify correctness
print(f"\n  Top 5 merchants by payout:")
final.orderBy(desc("total_payout")).show(5)


FIX 3: Salted GroupBy
  Fix 3 time:    8.572s (was 26.826s)
  Improvement:   68.0%

  Top 5 merchants by payout:
+-----------+---------------+--------+------------------+-------+------------------+
|merchant_id|  merchant_name|    tier|      total_payout|streams|      avg_duration|
+-----------+---------------+--------+------------------+-------+------------------+
| MEGA_MUSIC|Mega Music Inc.|platinum|1558.5005399999866| 399875|182.44444388871523|
|  BIG_BEATS|  Big Beats Co.|    gold| 284.4328300000003|  79626| 182.6783211513827|
| SOUND_SHOP| Sound Shop LLC|  silver|131.90574999999984|  40592|182.07023551438706|
| store_0027|       Store 27|  bronze|           2.13225|    649|175.57473035439136|
| store_0094|       Store 94|  bronze|2.0763000000000003|    636|181.79245283018867|
+-----------+---------------+--------+------------------+-------+------------------+
only showing top 5 rows


In [9]:
print("=" * 60)
print("FIX 4: AQE (Adaptive Query Execution)")
print("=" * 60)

spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

start = time.time()

events_valid = events.filter(col("merchant_id").isNotNull())
joined_aqe = events_valid.join(merchants, "merchant_id")

joined_aqe.groupBy("merchant_id", "merchant_name", "tier") \
    .agg(sum(col("revenue_per_stream") * col("payout_rate")), count("*"), avg("duration_sec")) \
    .collect()

t_fix4 = time.time() - start
print(f"  AQE time:      {t_fix4:.3f}s (was {baseline_total:.3f}s)")
print(f"  Improvement:   {(1 - t_fix4/baseline_total)*100:.1f}%")
print(f"  (No code changes! Just config)")

spark.conf.set("spark.sql.adaptive.enabled", "false")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "10485760")


FIX 4: AQE (Adaptive Query Execution)
  AQE time:      6.397s (was 26.826s)
  Improvement:   76.2%
  (No code changes! Just config)


In [10]:
print("=" * 60)
print("FULLY OPTIMIZED PIPELINE")
print("=" * 60)

spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")
spark.conf.set("spark.sql.shuffle.partitions", "8")

total_start = time.time()

# Step 1: Filter nulls
events_clean = events.filter(col("merchant_id").isNotNull())
events_nulls = events.filter(col("merchant_id").isNull())

# Step 2: Broadcast join with merchants
enriched = events_clean.join(broadcast(merchants), "merchant_id") \
    .withColumn("payout", col("revenue_per_stream") * col("payout_rate"))

# Step 3: Cache (reused for 3 reports)
enriched.cache()
cache_start = time.time()
row_count = enriched.count()
cache_time = time.time() - cache_start

# Report 1: Merchant revenue
r1 = enriched.groupBy("merchant_id", "merchant_name", "tier") \
    .agg(sum("payout").alias("total_payout"),
         count("*").alias("streams"),
         avg("duration_sec").alias("avg_duration")) \
    .orderBy(desc("total_payout"))

# Report 2: Genre revenue
r2 = enriched.groupBy("genre") \
    .agg(sum("revenue_per_stream").alias("total_rev"),
         count("*").alias("streams"))

# Report 3: Completion by tier
r3 = enriched.groupBy("tier") \
    .agg((sum(when(col("completed"), 1).otherwise(0)) / count("*") * 100).alias("completion_rate"),
         count("*").alias("total_streams"))

# Null summary
null_count = events_nulls.count()

# Collect all
r1.collect()
r2.collect()
r3.collect()

optimized_total = time.time() - total_start

print(f"  Cache:           {cache_time:.3f}s ({row_count:,} rows)")
print(f"  3 reports:       {optimized_total - cache_time:.3f}s")
print(f"  Null events:     {null_count:,} (handled separately)")
print(f"  TOTAL:           {optimized_total:.3f}s")
print(f"\n  BASELINE:        {baseline_total:.3f}s")
print(f"  SPEEDUP:         {baseline_total/optimized_total:.1f}x")
print(f"  TIME SAVED:      {baseline_total - optimized_total:.3f}s "
      f"({(1-optimized_total/baseline_total)*100:.0f}%)")

enriched.unpersist()
spark.conf.set("spark.sql.adaptive.enabled", "false")


FULLY OPTIMIZED PIPELINE
  Cache:           8.083s (639,864 rows)
  3 reports:       7.672s
  Null events:     160,136 (handled separately)
  TOTAL:           15.756s

  BASELINE:        26.826s
  SPEEDUP:         1.7x
  TIME SAVED:      11.070s (41%)


In [11]:
print("=" * 65)
print("OPTIMIZATION RESULTS SUMMARY")
print("=" * 65)
print(f"{'Technique':<40} {'Time':>8} {'Speedup':>10}")
print("-" * 65)
print(f"{'Baseline (all anti-patterns)':<40} {baseline_total:>7.3f}s {'1.0x':>10}")
print(f"{'+ Null key handling':<40} {t_fix1:>7.3f}s {baseline_total/t_fix1:>9.1f}x")
print(f"{'+ Broadcast join + cache':<40} {t_fix2:>7.3f}s {baseline_total/t_fix2:>9.1f}x")
print(f"{'+ Salted groupBy':<40} {t_fix3:>7.3f}s {baseline_total/t_fix3:>9.1f}x")
print(f"{'+ AQE only (no code changes)':<40} {t_fix4:>7.3f}s {baseline_total/t_fix4:>9.1f}x")
print(f"{'All combined (production)':<40} {optimized_total:>7.3f}s {baseline_total/optimized_total:>9.1f}x")
print("=" * 65)


OPTIMIZATION RESULTS SUMMARY
Technique                                    Time    Speedup
-----------------------------------------------------------------
Baseline (all anti-patterns)              26.826s       1.0x
+ Null key handling                        9.612s       2.8x
+ Broadcast join + cache                  14.597s       1.8x
+ Salted groupBy                           8.572s       3.1x
+ AQE only (no code changes)               6.397s       4.2x
All combined (production)                 15.756s       1.7x


StreamPulse Revenue Pipeline — Optimization Analysis
🔍 Skew Analysis

MEGA_MUSIC accounts for ~50% of 800K events (~400K rows)

BIG_BEATS ~10%

SOUND_SHOP ~5%

200 small stores → very low frequency

~20% of events have NULL merchant_id

🚨 Problem Identified

Severe key skew on merchant_id

One shuffle partition receives ~50% of the data

Causes:

Straggler tasks

Long stage completion

Poor CPU utilization

4× slower pipeline

📉 Baseline (Unoptimized)
Issues Present

Null keys included in join

Broadcast disabled

AQE disabled

Forced SortMergeJoin

Skewed groupBy on merchant_id

No caching

Repeated recomputation of joins

Result

Large shuffle during join

Uneven partition distribution

One partition dominating execution time

Overall slow pipeline

🥇 Biggest Single Impact
✅ Broadcast Join (+ Caching)
Why It Helped Most

Merchant table has only 203 rows

Without broadcast → full shuffle of 800K rows

With broadcast:

Small table sent to each executor

No shuffle required

Join becomes map-side

Why Caching Helped

Joined dataset reused for 3 reports

Without cache → recomputation

With cache → reused from memory

🔥 Conclusion: Broadcast join + cache produced the largest performance gain.

🧂 When to Use Salting vs AQE
Use AQE When:

Running Spark 3.x+

Want minimal code changes

Data patterns vary

Production cluster environment

Prefer automatic skew handling

Use Salting When:

Running older Spark (no AQE)

Skew is extreme (70–90% single key)

Need deterministic behavior

AQE not allowed in environment

AQE fails to mitigate skew sufficiently

⚙️ Production Partition Configuration
Shuffle Partitions

Set to 2–3× total executor cores

Example:

100 total cores → 200–300 shuffle partitions

Enable AQE

spark.sql.adaptive.enabled = true

spark.sql.adaptive.skewJoin.enabled = true

spark.sql.adaptive.coalescePartitions.enabled = true

Broadcast Threshold

Increase threshold if memory allows

Ensure small dimension tables are broadcast

Target Partition Size

128MB – 256MB per partition

Avoid:

Too many tiny partitions

Very large partitions (>1GB)

🏗️ Changes on a 100-Node Cluster

Assume:

100 nodes

8 cores per node

800 total cores

Adjustments

Shuffle partitions → 1600–2400

AQE strongly recommended

Broadcast still required

Salting may not be necessary due to higher parallelism

Cache using MEMORY_AND_DISK if needed

🧠 Root Causes of 4× Slowdown

SortMergeJoin on skewed key

50% of data in single merchant key

Null keys increasing wasted processing

No caching for repeated aggregations

Default shuffle configuration

🚀 Final Production Design
Configuration

AQE enabled

Skew join enabled

Shuffle partitions = 2–3× cores

Broadcast small dimension tables

Cache reused datasets

Code Strategy

Filter null keys early

Broadcast small lookup tables

Cache reused intermediate datasets

Use AQE for automatic skew handling

Apply salting only if skew persists

📌 Final Takeaways

Skew is a data distribution problem, not a hardware problem

Broadcast is the most powerful optimization for small dimensions

AQE is essential in modern Spark production workloads

Salting is a targeted solution for extreme skew

Caching matters when dataset reuse > 1